# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [54]:
import pandas as pd
import numpy as np
from scipy import stats


In [9]:
#Your code here
df = pd.read_csv('homepage_actions.csv')
df

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view


In [47]:
duplicates = df.duplicated(subset=['id', 'action'])
duplicates.any()

np.False_

In [41]:
action_unique = df['action'].unique()
action_unique

array(['view', 'click'], dtype=object)

In [42]:
# how many viewers also clicked?
clicked = df[df.action == 'click']['id'].unique()
number_of_clickers = len(clicked)
number_of_clickers

1860

In [43]:
# did anyone click who did not view?
viewed = df[df['action'] == 'view']['id'].unique()
number_of_viewers = len(viewed)
number_of_viewers

6328

In [44]:
control = df[df['group'] == 'control']['id']
experiment = df[df['group'] == 'experiment']['id']

print(len(control), len(experiment))

4264 3924


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [61]:
H_0 = 'The new homepage does not increase clicks.'
H_1 = 'The new homepage does increase clicks.'

In [49]:
#Your code here
control_views = len(df[(df['group'] == 'control') & (df['action'] == 'view')])
control_clicks = len(df[(df['group'] == 'control') & (df['action'] == 'click')])
experiment_views = len(df[(df['group'] == 'experiment') & (df['action'] == 'view')])
experiment_clicks = len(df[(df['group'] == 'experiment') & (df['action'] == 'click')])

p1 = experiment_clicks / experiment_views
p2 = control_clicks / control_views

print(p1, p2)


0.3097463284379172 0.2797118847539016


In [52]:
p_pooled = (experiment_clicks + control_clicks) / (experiment_views + control_views)
se = np.sqrt(p_pooled * (1 - p_pooled) * (1 / experiment_views + 1 / control_views))
z_stat = (p1 - p2) / se
z_stat

np.float64(2.618563885349469)

In [56]:
p_val = 1 - stats.norm.cdf(z_stat)
p_val

np.float64(0.004415037788297882)

In [57]:
alpha = 0.05
if p_val < alpha:
  print('Null hypothesis rejected.')
  print('The experimental homepage is more effective than the control group.')
else:
  print('Failed to reject the null hypothesis.')
  print('There is not enough evidence to conclude the experimental homepage is more effective than that of the control group.')

Null hypothesis rejected.
The experimental homepage is more effective than the control group.


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [58]:
#Your code here
expected_experiment_clicks = experiment_views * p2
expected_experiment_clicks

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [59]:
#Your code here
var = experiment_views * p2 * (1 - p2)
std = np.sqrt(var)
z_score = (experiment_clicks - expected_experiment_clicks) / std
z_score

np.float64(3.6625360854823588)

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [60]:
#Your code here
p_val = 1 - stats.norm.cdf(z_score)
p_val

np.float64(0.00012486528006949715)

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: The p_values vary slightly but the conclusion remains that we reject the null hypothesis.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.